<a href="https://colab.research.google.com/github/isadays/Lindblad_equation/blob/dev/Lindblad_equationdev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I want to integrate classical methods to optimize hyperparameters from quantum world.